In [2]:
# ==============================================================================
# Importing Library
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.document_loaders import Docx2txtLoader
import threading
from langchain_core.pydantic_v1 import BaseModel, Field, validator, root_validator
from langchain_community.document_loaders import FireCrawlLoader
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
import os
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from typing import List
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
# from src.models import ResponseModel, SentenceParser
from langchain.chains import LLMChain
import concurrent.futures
from difflib import ndiff
import pandas as pd

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv('../src/.env')

True

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.batch(
    [{
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    },
    {
        "input_language": "English",
        "output_language": "urdu",
        "input": "I love programming.", 
    }]
)

[AIMessage(content='Ich liebe Programmieren.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 26, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4eb04b9e-84dc-4686-9a94-1e66472071de-0', usage_metadata={'input_tokens': 26, 'output_tokens': 5, 'total_tokens': 31}),
 AIMessage(content='مجھے پروگرامنگ کرنا پسند ہے۔', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 27, 'total_tokens': 55}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-77b8fb6f-d798-46ae-991c-1a5c5254bbaa-0', usage_metadata={'input_tokens': 27, 'output_tokens': 28, 'total_tokens': 55})]

In [1]:
sentences = [
    {"input_language": "English", "output_language": "urdu", "input": "I love programming."},
    {"input_language": "English", "output_language": "urdu", "input": "The weather is nice today."},
    {"input_language": "English", "output_language": "urdu", "input": "What time is it?"}
]

# Extract the sentences for batching
inputs = [sentence["input"] for sentence in sentences]
input_text = "\n".join(inputs)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

# Define the chain
chain = prompt | llm

# Batch request
response = chain.invoke({
    "input_language": "English",
    "output_languag":"urdu",
    "input": "Hello, How are you"})

NameError: name 'ChatPromptTemplate' is not defined

In [18]:
response

AIMessage(content='مجھے پروگرامنگ کرنا پسند ہے۔\nآج موسم خوبصورت ہے۔\nوقت کیا ہوا ہے؟', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 38, 'total_tokens': 106}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bd47b4ac-5e9a-433c-b752-fad14297899c-0', usage_metadata={'input_tokens': 38, 'output_tokens': 68, 'total_tokens': 106})